# 7.1 전체 구조 (CNN)
CNN의 네트워크 구조를 살펴보며 전체 틀을 이해해보자<BR>
CNN도 지금까지 본 신경망과 같이 레고 블록처럼 계층을 조합항 만들 수 있다. 다만 **합성곱 계층(convolutional layer)** 과 **풀링 계층(pooling layer)** 가 새롭게 등장한다. <br>
<BR>
지금까지 본 신경망은 인접하는 계층의 모든 뉴런과 결합되어 있었다. 이를 **완전연결(fully-connected, 전결합)** 이라고 하며, 완전히 연결된 계층을 **Affine 계층** 이라는 이름으로 구현했다.

Affine 계층을 사용하면, 가령 층이 5개인 완전연결 신경망은 아래와 같이 구현할 수 있다.<Br><BR>
<img src="https://velog.velcdn.com/images/jw5150/post/404b1898-f129-45f2-a5c5-4905f1a74f3a/image.png" width=1200 height=250></img><br><br> 
위와 같이 완전연결 신경망은 Affine 계층 뒤에 활성화 함수를 갖는 ReLU 계층(혹은 Sigmoid 계층)이 이어진다. 그럼 CNN 구조는 어떨까?

<img src="https://velog.velcdn.com/images/jw5150/post/b1f040df-5031-4ea2-a3df-8792ef545406/image.png" width=1200 height=250></img><br><br> 
위 그림과 같이 CNN에서는 새로운 '합성곱 계층(Conv)'과 '풀링 계층(Pooling)'이 추가된다. CNN의 계층은 'Conv-ReLU-(Pooling)' 흐름으로 연결된다(풀링 계층 생략하기도 함).<BR>
지금까지 'Affine-ReLU' 연결이 'Conv-ReLU-(Pooling)'으로 바뀌었다 생각

위 그림에서 또 주목할 점은 출력에 가까운 층에서는 지금까지의 'Affine-ReLU' 구성을 사용할 수 있다는 것이다. 마지막 출력 계층에서는 'Affine-Softmax' 조합을 그대로 사용한다.

# 7.2 합성곱 계층
CNN 에서는 **패딩(padding), 스트라이드(stride)** 등 CNN 고유의 용어가 등장한다. 또 각 계층 사이에는 3차원 데이터같이 입체적인 데이터가 흐른다는 점에서 완전연결 신경망과 다르다.

## 7.2.1 완전연결 계층의 문제점
지금까지 완전연결 계층(Affine 계층)을 사용했다. 완전연결 계층에서는 인접하는 계층의 뉴런이 모두 연결되고 출력의 수는 임의로 정할 수 있다. <bR>
<Br>
완전연결 계층의 문제점은 무엇일까? 바로 '데이터의 형상이 무시'된다는 사실이다. 입력 데이터가 이미지인 경우를 예로 들면, 이미지는 통상 세로, 가로, 채널(색상)로 구성된 3차원 데이터이다. 그러나 완전연결 계층에 입력할 때는 3차원 데이터를 평평한 1차원 데이터로 평탄화해줘야 한다. 지금까지 MNIST 데이터셋을 사용한 사례에서는 형상이 (1, 28, 28)인 이미지(1채널, 세로 28픽셀, 가로 28픽셀)를 1줄로 세운 784개의 데이터를 첫 Affine 계층에 입력했다. <BR><BR>
이미지는 3차원 형상이며, 이 형상에는 소중한 공간적 정보가 담겨있다. 예를 들어 공간적으로 가까운 픽셀은 값이 비슷하거나, RGB의 각 채널은 서로 밀접하게 관련되어 있거나, 거리가 먼 픽셀끼리는 별 연관이 없는 등, 3차원 속에서 의미를 갖는 본질적인 패턴이 숨어 있을 것이다. 그러나 완전연결 계층은 형상을 무시하고 모든 입력 데이터를 동등한 뉴런(같은 차원의 뉴런)으로 취급하여 형상에 담긴 정보를 살릴 수 없다.<Br><BR>
한편, 합성곱 계층은 형상을 유지한다. 이미지도 3차원 데이터로 입력받으며, 마찬가지로 다음 계층에도 3차원 데이터로 전달한다. 그래서 CNN에서는 이미지처럼 형상을 가진 데이터를 제대로 이해할(가능성이 있는) 것이다.<br><BR>
CNN에서는 합성곱 계층의 입출력 데이터를 **특징 맵(feature map)** 이라고도 한다. 합성곱 계층의 입력 데이터를 **입력 특징 맵(input feature map)** 출력 데이터를 **출력 특징 맵(output feature map)** 이라고 하는 식이다. 이 책에서는 '입출력 데이터'와 '특징 맵'을 같은 의미로 사용한다.

# 7.2.2 합성곱 연산
합성곱 계층에서의 **합성곱 연산** 을 처리한다. 합성곱 연산은 이미지 처리에서 말하는 **필터 연산** 에 해당한다. 

<img src="https://velog.velcdn.com/images/jw5150/post/9d011f29-cb94-4b00-9510-27e9399b53b6/image.png"></img><br><br> 

위 그림과 같이 합성곱 연산은 입력 데이터에 필터를 적용한다. 입력 데이터는 세로, 가로 방향의 형상을 가졌고, 필터 역시 마찬가지이다. 데이터와 필터의 형상을 (높이 height, 너비 width) 로 표기하며, 위 예에서는 입력 (4, 4), 필터 (3, 3), 출력 (2, 2)가 된다. 필터를 **커널** 이라고도 한다.

<img src="https://velog.velcdn.com/images/jw5150/post/775209a0-9cf9-4157-b9bb-099cd6a4486e/image.png"></img><br><br> 

합성곱 연산은 윈도우(회색 영역)를 일정 간격(stride)으로 이동하며 원소끼리 곱하고 그 총합을 구한다(이를 단일 곱셈-누산 fused multiply-add, FMA라 한다). 그리고 그 결과를 해당 장소에 저장하고 이 과정을 모든 장소에서 수행하면 합성곱 연산의 출력이 완성된다.

완전연결 신경망에서는 가중치 매개변수와 편향이 존재하는데, CNN에서는 필터의 매개변수가 그동안의 '가중치'에 해당한다. 그리고 CNN에서도 편향이 존재한다. 

<img src="https://velog.velcdn.com/images/jw5150/post/a5a7b944-c860-416b-9c36-7540b63ce379/image.png"></img><br><br> 

편향까지 포함하면 위 그림과 같은 흐름이 된다. 편향은 필터를 적용한 후의 데이터에 더해진다. 그리고 편향은 항상 하나(1 x 1) 만 존재한다. 그 하나의 값을 필터를 적용한 모든 원소에 더한다.

## 7.2.3 패딩
합성곱 연산을 수행하기 전에 입력 데이터 주변을 특정 값(예컨대 0)으로 채우기도 한다. 이를 **패딩(padding)** 이라고 한다. 예를 들어 아래 그림은 (4, 4) 크기의 입력 데이터에 폭이 1인 패딩을 적용한 모습이다. 폭 1짜리 패딩이라 하면 입력 데이터 사방 1픽셀을 특정 값으로 채운다.

<img src="https://velog.velcdn.com/images/jw5150/post/e85f8ac6-35d6-4b78-8df9-4156162e7b06/image.png"></img><br><br> 

위 그림 같이 처음에 크기가 (4, 4)인 입력 데이터에 패딩이 추가되어 (6, 6)이 된다. 이 입력에 (3, 3) 크기의 필터를 걸면 (4, 4) 크기의 출력 데이터가 생성된다. 이 예에서는 패딩을 1로 설정했지만, 2나 3 등 원하는 정수로 설정할 수 있다. 

**NOTE_** 패딩은 주로 출력 크기를 조정할 목적으로 사용된다. 예를 들어 (4, 4) 입력 데이터에 (3, 3) 필터를 적용하면 출력은 (2, 2)가 되어, 입력보다 2만큼 줄어든다. 이는 합성곱 연산을 몇 번이나 되풀이하는 심층 신경망에서 문제가 될 수 있다. 합성곱 연산을 거칠 때마다 크기가 작아지면 어느 시점에서는 출력 크기가 1이 되어버린다. 더 이상은 합성곱 연산을 진행할 수 없게 된다. 이러한 사태를 막기 위해 패딩을 사용한다. 앞의 예에서는 패딩의 폭을 1로 설정하니 (4, 4) 입력에 대한 출력이 같은 크기인 (4, 4)로 유지되었다. 한 마디로 입력 데이터의 공간적 크기를 고정한 채로 다음 계층에 전달할 수 있다.

## 7.2.4 스트라이드
필터를 적용하는 위치의 간격을 **스트라이드(stride)** 라고 한다. 아래는 스트라이드를 2로 한 그림이다. 필터를 적용하는 윈도우가 두 칸씩 이동한다.

<img src="https://velog.velcdn.com/images/jw5150/post/473e3a2d-cbe3-4f3c-b73a-30dc488fb7b9/image.png"></img><br><br> 

위 그림에서는 크기가 (7, 7)인 입력 데이터에 스트라이드를 2로 설정한 필터를 적용한다. 이처럼 스트라이드는 필터를 적용하는 간격을 지정한다.

패딩이 커지면 출력의 크기가 커지고, 스트라이드가 커지면 출력의 크기가 작아진다. 이를 수식화하면 다음과 같다.

<img src="https://velog.velcdn.com/images/jw5150/post/78082213-e523-4aea-bb50-f537cf949977/image.png"></img><br><br> 

- 입력 크기: (H, W)
- 필터 크기: (FH, FW)
- 출력 크기: (OH, HW)
- 패딩: P
- 스트라이드: S

위 식은 정수로 나누어떨어져야 한다. 출력 크기가 정수가 아니면 오류를 내는 등의 대응을 해줘야한다. 덧붙여서, 딥러닝 프레임워크 중에는 값이 딱 나눠떨어지지 않을 때는 가장 가까운 정수로 반올림하는 등, 특별히 에러를 내지 않고 진행하도록 구현하는 경우도 있다.

## 7.2.5 3차원 데이터의 합성곱 연산

아래는 3차원 데이터의 합성곱 연산 예이다.

<img src="https://velog.velcdn.com/images/jw5150/post/f9164b15-c17a-4eb4-9768-cf1720b90452/image.png"></img><br><br> 

2차원일 때와 비교하면, 길이 방향(채널 방향)으로 특징 맵이 늘어났다. 채널 쪽으로 특징 맵이 여러 개 있다면 입력 데이터와 필터의 합성곱 연산을 채널마다 수행하고, 그 결과를 더해서 하나의 출력을 얻는다.

<img src="https://velog.velcdn.com/images/jw5150/post/7455566b-73c5-4b18-bcb8-942d1b513a99/image.png"></img><br><br> 

주의할 점은 입력 데이터의 채널 수와 필터의 채널 수가 같아야 한다는 것이다. 이 예에서는 모두 3개로 일치한다. 한편, 필터 자체의 크기는 원하는 값으로 설정할 수 있다(단 모든 채널의 필터가 같은 크기여야 한다). 이 예에서는 필터의 크기가 (3, 3)이지만 원한다면 바꿀 수 있다. 

## 7.2.6 블록으로 생각하기
3차원의 합성곱 연산은 데이터와 필터를 직육면체 블록이라고 생각하면 쉽다. 

<img src="https://velog.velcdn.com/images/jw5150/post/5222c5b3-304d-4674-a9bc-7b4b2f13a22b/image.png"></img><br><br> 

이 예에서 출력 데이터는 한 장의 특징 맵이다. 한 장의 특징 맵을 다른 말로 하면 채널이 1개인 특징 맵이다. 그럼 합성곱 연산의 출력으로 다수의 채널을 내보내려면 어떻게 해야할까?<bR>
필터(가중치)를 다수 사용하는 것이다.

<img src="https://velog.velcdn.com/images/jw5150/post/657705dc-132d-4c1c-92e3-360745912509/image.png"></img><br><br> 

필터를 FN개(필터의 개수) 적용하면 출력 맵도 FN개가 생성된다. 그리고 그 FN개의 맵을 모으면 형상이 (FN, OH, OW)인 블록이 완성된다. 이 완성된 블록을 다음 계층으로 넘기겠따는 것이 CNN의 처리 흐름이다.

합성곱 연산에서는 필터의 수도 고려해야 한다. 그런 이유로 필터의 가중치 데이터는 4차원 데이터이며 (출력 채널 수, 입력  채널 수, 높이, 너비) 순으로 쓴다. 

예를 들어, 채널 수가 3, 크기가 (5, 5)인 필터가 20개면 (20, 3, 5, 5)로 쓴다.

<img src="https://velog.velcdn.com/images/jw5150/post/79186c7a-425d-4466-b02f-6333a72f4554/image.png"></img><br><br> 

편향은 채널 하나에 값 하나씩으로 구성된다. 이 예에서는 편향의 현상은 (FN, 1, 1)이고, 필터의 출력 결과의 형상은 (FN, OH, OW)이다. 이 두 블록을 더하면 편향의 각 값이 필터의 출력인 (FN, OH, OW) 블록의 대응 채널의 원소 모두에 더해진다.

## 7.2.7 배치 처리
신경망 처리에서는 입력 데이터를 한 덩어리로 묶어 배치로 처리했다. 완전연결 신경망을 구현하면서는 이 방식을 지원하여 처리 효율을 높이고, 미니배치 방식의 학습도 지원하도록 했다.<br>
<br>
합성곱 연산도 마찬가지로 배치 처리를 지원한다. 각 계층을 흐르는 데이터의 차원을 하나 늘려 4차원 데이터로 저장한다. 구체적으로는 데이터를 (데이터 수, 채널 수, 높이, 너비) 순으로 저장한다. 데이터가 N개일 때 위 그림을 배치 처리한다면 데이터 형태가 아래처럼 된다.

<img src="https://velog.velcdn.com/images/jw5150/post/ab374e7a-06d2-48b3-83df-dacdb0f04b6e/image.png"></img><br><br> 

배치 처리 시의 데이터 흐름을 나타낸 위 그림을 보면 각 데이터의 선두에 배치용 차원을 추가한다. 이처럼 데이터는 4차원 형상을 가진 채 각 계층을 타고 흐른다. 여기에서 주의할 점으로는 신경망에 4차원 데이터가 하나 흐를 때마다 데이터 N개에 대한 합성곱 연산이 이뤄진다는 것이다. 즉, N회 분의 처리를 한 번에 수행한다.

## 7.3 풀링 계층
풀링은 세로, 가로 방향의 공간을 줄이는 연산이다. 예를 들어 아래와 같이 2 x 2 영역을 원소 하나로 집약하여 공간 크기를 줄인다.

<img src="https://velog.velcdn.com/images/jw5150/post/a4226fa1-03b8-4f9f-baf1-02dbabd6e7d1/image.png"></img><br><br> 

위 그림은 2 x 2 **최대 풀링(max pooling)** 을 스트라이드 2로 처리하는 순서이다. 최대 풀링은 최댓값(max)을 구하는 연산으로, '2 x 2'는 대상 영역의 크기를 뜻한다. 즉 2 x 2 최대 풀링은 그림과 같이 2 x 2 크기의 영역에서 가장 큰 원소를 하나를 꺼낸다. 또, 스트라이드는 이 예에서는 2로 설정했으므로 2 x 2 윈도우가 원소 2칸 간격으로 이동한다. 참고로, 풀링의 윈도우 크기와 스트라이드는 같은 값으로 설정하는 것이 보통이다. 예를 들어 윈도우가 3 x 3이면 스트라이드는 3, 4 x 4이면 스트라이드를 4로 설정한다.

**WARNING_** 풀링은 최대 풀링 외에도 평균 풀링 등이 있따. 최대 풀링은 대상 영역에서 최댓값을 취하는 연산인 반면, 평균 풀링은 대상 영역의 평균을 계산한다. 이미지 인식 분야에서는 주로 최대 풀링을 사용한다.

## 7.3.1 풀링 계층의 특징
1. 학습해야 할 매개변수가 없다<br><BR>
풀링 계층은 합성곱 계층과 달리 학습해야 할 매개변수가 없다. 풀링은 대상 영역에서 최댓값이나 평균을 취하는 명확한 처리이므로 특별히 학습할 것이 없다.<BR><BR>
2. 채널 수가 변하지 않는다<BR><BR>
풀링 연산은 입력 데이터의 채널 수 그대로 출력 데이터로 내보낸다. 아래처럼 채널마다 독립적으로 계산하기 때문이다.<BR><BR>
<img src="https://velog.velcdn.com/images/jw5150/post/c8c07f1d-d011-4ccf-9f6f-776941872b8a/image.png" width=800 height=400></img><br><br> 
3. 입력의 변화에 영향을 적게 받는다(강건하다)<BR><BR>
입력 데이터가 조금 변해도 풀링의 결과는 잘 변하지 않는다. 예를 들어 아래에서 입력 데이터의 차이(데이터가 오른쪽으로 1칸씩 이동)를 풀링이 흡수해 사라지게 하는 모습을 보여준다.<Br><Br>
<img src="https://velog.velcdn.com/images/jw5150/post/efa2d4c0-1055-4af0-8e03-99dd5373b479/image.png" width=1000 height=250></img><br><br> 
입력 데이터가 가로로 1원소만큼 어긋나도 출력은 같다(데이터에 따라서는 다를 수도 있다)

## 7.4 합성곱/풀링 계층 구현하기